In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

In [2]:
import src.load as load
import src.dataset as ds
import numpy as np
import pandas as pd
import torch
import plotly.graph_objects as go
import matplotlib.pyplot as plt



In [3]:
import src.load as load
from src.dataset import HarvestDataset

train_dataset, test_dataset, mapping_dict, reverse_mappings, meta  = load.separate_prop('../data/processed/')


In [6]:
meta

,TransplantDate,Parcel,Lot,ProducerCode,Variety,Ha,Class,Type,WeekTransplanted,WeekCos,WeekSin,Year,ClimateSeries
4,2013-02-15,Z18,2,ZJL,Olivia,0.5400,BSUF,Uva Roja,7,0.663123,0.748511,2013,"[[0.3125368926, -0.0596579057, 0.0, 0.0, -0.54..."
10,2013-02-20,Z18,5,ZJL,Shiren,0.2700,CHE,Cherry Rojo,8,0.568065,0.822984,2013,"[[-1.020167475, -0.1291892644, 0.2, 0.2, -0.92..."
16,2013-02-23,Z17,8,ZJL,Amaí,0.5063,BSUF,Uva Roja,8,0.568065,0.822984,2013,"[[-0.1651137121, -0.949659491, 0.0, 0.0, 0.007..."
23,2013-03-01,Z17,11,ZJL,Baby Black Plum,0.0900,CHE,Heirloom Cherry,9,0.464723,0.885456,2013,"[[0.7783938882, 0.42706146570000003, 0.0, 0.0,..."
25,2013-03-01,Z17,11,ZJL,Habanero Tangerina,0.0675,CHE,Heirloom Cherry,9,0.464723,0.885456,2013,"[[0.7783938882, 0.42706146570000003, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,2024-07-18,V02,37,VAP,King,0.6300,BSUF,Uva Roja,29,-0.935016,-0.354605,2024,"[[0.4868501577, 0.3822832708, 0.0, 0.0, 0.2966..."
3785,2024-07-25,S12,4,SGB,Top 2323,0.1530,CHE,Cherry Rosa,30,-0.885456,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3786,2024-07-25,S12,4,SGB,Yoyomo,0.1020,BSUF,Uva Amarilla,30,-0.885456,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3788,2024-07-29,S12,8,SGB,King,0.0306,BSUF,Uva Roja,31,-0.822984,-0.568065,2024,"[[0.3676142667, 0.3320816298, 0.0, 0.0, 0.5941..."


In [9]:
test_dataset.features[:,0]

tensor([0.5400, 0.2700, 0.5063, 0.0900, 0.0675, 0.1808, 0.1480, 0.8160, 0.1280,
        0.1040, 0.1176, 0.2016, 0.3024, 0.0810, 0.0690, 0.0320, 0.0320, 0.0320,
        0.3360, 0.0320, 0.0320, 0.4800, 0.2240, 0.6080, 0.2560, 0.1440, 0.0540,
        0.1800, 0.1800, 0.3312, 0.3888, 0.0288, 0.0513, 0.0684, 0.0684, 0.3078,
        0.2394, 0.2736, 0.0171, 0.3840, 0.2240, 0.2052, 0.0770, 0.1026, 0.0684,
        0.0855, 0.1625, 0.0720, 0.4536, 0.2907, 0.3249, 0.2223, 0.4000, 0.6840,
        0.0516, 0.1376, 0.0900, 0.7088, 0.0225, 0.1013, 0.7763, 0.1238, 0.0900,
        0.0900, 0.7650, 0.0567, 0.0891, 0.0162, 0.7104, 0.0980, 0.1674, 0.0432,
        0.0648, 0.0432, 0.1710, 0.5760, 0.7632, 0.0648, 0.3892, 0.0720, 0.1224,
        0.0504, 0.3078, 0.1368, 0.1458, 0.2394, 0.2052, 0.1283, 0.1026, 0.0257,
        0.2736, 0.0513, 0.7353, 0.3591, 0.1368, 0.1260, 0.1026, 0.1197, 0.0513,
        0.2736, 1.1502, 0.9180, 0.1458, 0.1140, 0.5092, 0.3080, 0.1540, 0.0480,
        0.1680, 0.7988, 0.6638, 0.0900, 

***Production Plan***

In [9]:
meta['Ranch'] = meta['ProducerCode']

In [10]:

agg_cols = {'Ha':'sum',
    'WeekTransplanted':['min','max','nunique']}

summary_ranch = meta.groupby(['Ranch','Class','Type']).agg(agg_cols)
summary_class = meta.groupby(['Class','Type']).agg(agg_cols)
summary_class_ranch = meta.groupby(['Class','Type','Ranch']).agg(agg_cols)


ranch_pivot = meta.pivot_table(
index=['Ranch', 'Class', 'Type'],
columns='WeekTransplanted',
values='Ha',
aggfunc='sum',
fill_value=0,
)

class_pivot = meta.pivot_table(
    index=['Class', 'Type'],
    columns='WeekTransplanted',
    values='Ha',
    aggfunc='sum',
    fill_value=0,
)


In [16]:
summary_ranch

Ha WeekTransplanted            
                                 sum              min max nunique
Ranch Class Type                                                 
MAP   BSUF  Uva Roja          0.0640               14  14       1
      CHE   Cherry Naranja    0.1158               14  25       2
OAP   BSUF  Uva Amarilla      2.4354               16  29       7
            Uva Roja         13.7910               10  31      18
      CHE   Cherry Amarillo   0.2594               17  25       4
...                              ...              ...  ..     ...
ZJL   CHE   Cherry Rosa       0.0225                9   9       1
            Heirloom Cherry   1.0631                5  35       4
            Pera              0.4878                5  32       4
            Pera Amarilla     0.3038               33  35       2
            Pera Roja         0.2850               39  39       1

[95 rows x 4 columns]

In [17]:
summary_class

Ha WeekTransplanted            
                            sum              min max nunique
Class Type                                                  
BSUF  Uva Amarilla       8.0065               12  36      21
      Uva Roja          85.1094                5  35      30
      Uva Zebra          0.1040               19  21       2
CHE   Cherry Amarillo    1.0585               11  33      11
      Cherry Chocolate   0.9498               10  37       5
      Cherry Naranja     1.5583                6  32      12
      Cherry Rojo       36.5355                4  34      26
      Cherry Rosa        2.6922                9  33      14
      Heirloom Cherry   20.1770                5  36      27
      Pera               7.3167                5  32      24
      Pera Amarilla      5.9593                6  37      22
      Pera Roja          6.0043               10  39      21

In [14]:
ranch_pivot

WeekTransplanted              4       5      6    7    8       9       10  \
Ranch Class Type                                                            
MAP   BSUF  Uva Roja         0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   
      CHE   Cherry Naranja   0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   
OAP   BSUF  Uva Amarilla     0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   
            Uva Roja         0.0  0.0000  0.000  0.0  0.0  0.0000  0.2025   
      CHE   Cherry Amarillo  0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   
...                          ...     ...    ...  ...  ...     ...     ...   
ZJL   CHE   Cherry Rosa      0.0  0.0000  0.000  0.0  0.0  0.0225  0.0000   
            Heirloom Cherry  0.0  0.0767  0.000  0.0  0.0  0.0000  0.0000   
            Pera             0.0  0.2628  0.045  0.0  0.0  0.0675  0.0000   
            Pera Amarilla    0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   
            Pera Roja        0.0  0.0000  0.000  0.0  0.0  0.0000  0.0000   

WeekTransplanted              11      12    13  ...      29   30      31  \
Ranch Class Type                                ...                        
MAP   BSUF  Uva Roja         0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
      CHE   Cherry Naranja   0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
OAP   BSUF  Uva Amarilla     0.0  0.0000  0.00  ...  0.0810  0.0  0.0000   
            Uva Roja         0.0  0.3933  0.09  ...  0.0972  0.0  0.0648   
      CHE   Cherry Amarillo  0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
...                          ...     ...   ...  ...     ...  ...     ...   
ZJL   CHE   Cherry Rosa      0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
            Heirloom Cherry  0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
            Pera             0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
            Pera Amarilla    0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   
            Pera Roja        0.0  0.0000  0.00  ...  0.0000  0.0  0.0000   

WeekTransplanted                 32      33   34      35   36   37     39  
Ranch Class Type                                                           
MAP   BSUF  Uva Roja         0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
      CHE   Cherry Naranja   0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
OAP   BSUF  Uva Amarilla     0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
            Uva Roja         0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
      CHE   Cherry Amarillo  0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
...                             ...     ...  ...     ...  ...  ...    ...  
ZJL   CHE   Cherry Rosa      0.0000  0.0000  0.0  0.0000  0.0  0.0  0.000  
            Heirloom Cherry  0.3788  0.0788  0.0  0.5288  0.0  0.0  0.000  
            Pera             0.1125  0.0000  0.0  0.0000  0.0  0.0  0.000  
            Pera Amarilla    0.0000  0.0788  0.0  0.2250  0.0  0.0  0.000  
            Pera Roja        0.0000  0.0000  0.0  0.0000  0.0  0.0  0.285  

[95 rows x 35 columns]

In [15]:
summary_class_ranch

Ha WeekTransplanted            
                             sum              min max nunique
Class Type         Ranch                                     
BSUF  Uva Amarilla OAP    2.4354               16  29       7
                   OSF    0.8845               14  26       5
                   SAB    0.5940               15  30       4
                   SFB    1.1320               14  28       5
                   SGB    0.0204               18  18       1
...                          ...              ...  ..     ...
CHE   Pera Roja    SGC    0.3042               16  16       1
                   SJB    0.0552               24  24       1
                   SMB    0.0600               27  27       1
                   VAP    1.3180               10  30       6
                   ZJL    0.2850               39  39       1

[95 rows x 4 columns]

In [12]:
class_pivot

WeekTransplanted            4       5       6       7       8       9   \
Class Type                                                               
BSUF  Uva Amarilla      0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
      Uva Roja          0.0000  0.2795  0.4163  0.0000  1.5076  2.4517   
      Uva Zebra         0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
CHE   Cherry Amarillo   0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
      Cherry Chocolate  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
      Cherry Naranja    0.0000  0.0000  0.0540  0.0000  0.0000  0.0000   
      Cherry Rojo       0.4224  0.0000  0.0000  0.8439  1.0463  0.0000   
      Cherry Rosa       0.0000  0.0000  0.0000  0.0000  0.0000  0.0225   
      Heirloom Cherry   0.0000  0.0767  0.0000  0.0000  0.0000  0.0000   
      Pera              0.0000  0.2628  0.0450  0.0000  0.0000  0.0675   
      Pera Amarilla     0.0000  0.0000  0.0540  0.0000  0.0000  0.0000   
      Pera Roja         0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   

WeekTransplanted            10      11      12      13  ...      29      30  \
Class Type                                              ...                   
BSUF  Uva Amarilla      0.0000  0.0000  0.0288  0.1426  ...  0.9484  0.4743   
      Uva Roja          0.3649  4.1572  4.0560  5.7984  ...  2.3746  3.5287   
      Uva Zebra         0.0000  0.0000  0.0000  0.0000  ...  0.0000  0.0000   
CHE   Cherry Amarillo   0.0000  0.2604  0.0000  0.0480  ...  0.1884  0.0000   
      Cherry Chocolate  0.3200  0.0000  0.0000  0.0204  ...  0.0000  0.0000   
      Cherry Naranja    0.0000  0.0000  0.0000  0.0000  ...  0.0000  0.1805   
      Cherry Rojo       0.5592  2.5598  1.4912  1.6628  ...  2.6948  0.3340   
      Cherry Rosa       0.2720  0.0000  0.0000  0.1152  ...  0.0000  0.0000   
      Heirloom Cherry   0.4160  0.7239  0.9305  1.4423  ...  1.2278  0.9050   
      Pera              0.0000  0.3840  0.2940  0.1616  ...  0.3646  0.0000   
      Pera Amarilla     0.1944  0.3337  0.1710  0.0000  ...  0.4160  0.1020   
      Pera Roja         0.1600  0.0900  0.5294  0.4400  ...  0.2220  0.1272   

WeekTransplanted            31      32      33      34      35      36  \
Class Type                                                               
BSUF  Uva Amarilla      0.0960  0.0000  0.0000  0.0000  0.1200  0.1260   
      Uva Roja          1.7668  0.7313  0.9143  0.6477  1.5597  0.0000   
      Uva Zebra         0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
CHE   Cherry Amarillo   0.0000  0.0000  0.0320  0.0000  0.0000  0.0000   
      Cherry Chocolate  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
      Cherry Naranja    0.0000  0.1220  0.0000  0.0000  0.0000  0.0000   
      Cherry Rojo       0.1857  0.0000  0.0000  1.0066  0.0000  0.0000   
      Cherry Rosa       0.0480  0.0000  0.0800  0.0000  0.0000  0.0000   
      Heirloom Cherry   0.2992  0.8288  0.4588  0.0000  0.5288  0.0306   
      Pera              0.1980  0.1125  0.0000  0.0000  0.0000  0.0000   
      Pera Amarilla     0.0000  0.0000  0.5188  0.0000  0.2250  0.0000   
      Pera Roja         0.0000  0.0000  0.3000  0.0000  0.0000  0.0000   

WeekTransplanted            37     39  
Class Type                             
BSUF  Uva Amarilla      0.0000  0.000  
      Uva Roja          0.0000  0.000  
      Uva Zebra         0.0000  0.000  
CHE   Cherry Amarillo   0.0000  0.000  
      Cherry Chocolate  0.1440  0.000  
      Cherry Naranja    0.0000  0.000  
      Cherry Rojo       0.0000  0.000  
      Cherry Rosa       0.0000  0.000  
      Heirloom Cherry   0.0000  0.000  
      Pera              0.0000  0.000  
      Pera Amarilla     0.2304  0.000  
      Pera Roja         0.0000  0.285  

[12 rows x 35 columns]

In [24]:
data = []
for class_type in class_pivot.index:
    data.append(go.Bar(
        name=f'{class_type}',
        x=class_pivot.columns,
        y=class_pivot.loc[class_type]
    ))
  
# Create the figure
fig = go.Figure(data=data)

# Update layout for the plot
fig.update_layout(
    barmode='stack',
    title='Stacked Bar Plot of Hectares by Week Transplanted',
    xaxis_title='Week Transplanted',
    yaxis_title='Hectares (Ha)',
    legend_title='Class and Type',
    xaxis=dict(tickangle=45, categoryorder='total descending'),
    legend=dict(x=1.05, y=1)
)




In [25]:


# Prepare data for the stacked bar plot
ranch_data = []
for ranch in ranch_pivot.index:
    ranch_data.append(go.Bar(
        name=f'{ranch}',
        x=ranch_pivot.columns,
        y=ranch_pivot.loc[ranch]
    ))

# Create the figure
ranch_fig = go.Figure(data=ranch_data)

# Update layout for the plot
ranch_fig.update_layout(
    barmode='stack',
    title='Stacked Bar Plot of Hectares by Week Transplanted (Ranch)',
    xaxis_title='Week Transplanted',
    yaxis_title='Hectares (Ha)',
    legend_title='Ranch',
    xaxis=dict(tickangle=45, categoryorder='total descending'),
    legend=dict(x=1.05, y=1)
)


load in model

In [19]:
from src.model import KiloModel,ScheduleModel
from src.encoder import ClimateEncoder

In [27]:
_kilo_model_path = '../app/k_model.pth'
_schedule_model_path = '../app/s_model.pth'

In [28]:
kilo_encoder = ClimateEncoder()
schedule_encoder = ClimateEncoder()
kilo_model = KiloModel(kilo_encoder)
kilo_model.load_state_dict(torch.load(_kilo_model_path))
kilo_model.eval()

schedule_model = ScheduleModel(schedule_encoder)
schedule_model.load_state_dict(torch.load(_schedule_model_path))
schedule_model.eval()

/Users/zakir/anaconda3/envs/cherry/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning:

`torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.



ScheduleModel(
  (encoder): ClimateEncoder(
    (feature_encoder): Sequential(
      (0): Linear(in_features=5, out_features=32, bias=True)
      (1): ReLU()
    )
    (climate_gru): GRU(9, 32, num_layers=2, batch_first=True, dropout=0.2)
    (ranch_emb): Embedding(13, 12)
    (parcel_emb): Embedding(44, 12)
    (lot_emb): Embedding(75, 12)
    (lot_to_parcel): Linear(in_features=12, out_features=12, bias=True)
    (parcel_to_ranch): Linear(in_features=12, out_features=12, bias=True)
    (class_emb): Embedding(2, 12)
    (type_emb): Embedding(14, 12)
    (variety_emb): Embedding(59, 12)
    (type_to_class): Linear(in_features=12, out_features=12, bias=True)
    (variety_to_type): Linear(in_features=12, out_features=12, bias=True)
  )
  (kilo_gru): GRU(3, 32, num_layers=2, batch_first=True, dropout=0.2)
  (schedule_output): Sequential(
    (0): Linear(in_features=168, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [55]:
outputs = kilo_model(test_dataset.features,test_dataset.encoded_features,test_dataset.climate_data,test_dataset.Y_combined[:,:10,:])


In [60]:
preds = outputs.detach().numpy().sum(axis=1)

In [61]:
actuals = test_dataset.Y_kilos.detach().numpy().sum(axis=1)

In [62]:
preds

array([ 935.74097,  878.0376 ,  915.28326,  765.7602 ,  761.2605 ,
        769.20166,  706.2112 ,  812.471  ,  805.9408 ,  830.69116,
        788.1886 ,  700.7489 ,  902.9768 ,  800.2494 ,  685.3682 ,
        876.0423 ,  701.276  ,  770.58966,  761.77405,  712.16394,
        830.4024 ,  927.6593 ,  800.635  ,  793.0717 ,  903.3398 ,
        723.3081 ,  826.6186 ,  829.43604,  657.56494,  753.44104,
        782.873  ,  777.505  ,  867.4995 ,  671.6677 ,  807.08167,
        777.6943 ,  718.02826,  873.91504,  907.5198 ,  747.87305,
        700.2075 ,  868.6288 ,  802.22577,  712.5901 ,  873.18005,
        690.64075,  899.07007,  678.55774,  780.4724 ,  653.56744,
        780.4886 ,  612.9413 ,  750.6034 ,  750.6206 ,  734.2047 ,
        801.8231 ,  872.32043,  908.91113,  650.2723 ,  752.3915 ,
        698.5276 , 1002.95703,  787.2033 ,  871.6021 ,  890.4129 ,
       1018.7187 ,  888.373  ,  956.8069 ,  845.0968 ,  889.5714 ,
        985.32513,  801.65106,  975.82007, 1019.9341 ,  891.24

In [65]:
meta

,TransplantDate,Parcel,Lot,ProducerCode,Variety,Ha,Class,Type,WeekTransplanted,WeekCos,WeekSin,Year,ClimateSeries,Ranch
2,2013-02-13,Z18,6,ZJL,Shiren,0.3938,CHE,Cherry Rojo,7,0.663123,0.748511,2013,"[[-0.2535675497, -0.8592687109, 0.0, 0.0, -0.1...",ZJL
5,2013-02-15,Z18,5,ZJL,Felicity,0.2138,CHE,Cherry Rojo,7,0.663123,0.748511,2013,"[[0.3125368926, -0.0596579057, 0.0, 0.0, -0.54...",ZJL
10,2013-02-20,Z18,5,ZJL,Shiren,0.2700,CHE,Cherry Rojo,8,0.568065,0.822984,2013,"[[-1.020167475, -0.1291892644, 0.2, 0.2, -0.92...",ZJL
16,2013-02-23,Z17,8,ZJL,Amaí,0.5063,BSUF,Uva Roja,8,0.568065,0.822984,2013,"[[-0.1651137121, -0.949659491, 0.0, 0.0, 0.007...",ZJL
17,2013-02-23,Z17,8,ZJL,Amsterdam,0.1463,BSUF,Uva Roja,8,0.568065,0.822984,2013,"[[-0.1651137121, -0.949659491, 0.0, 0.0, 0.007...",ZJL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3762,2024-07-08,S07,11,SFB,Pera Amarilla Viv,0.0400,CHE,Pera Amarilla,28,-0.970942,-0.239316,2024,"[[0.37274458930000004, 0.539771659, 0.0, 0.0, ...",SFB
3764,2024-07-08,S07,11,SFB,Top 2323,0.0800,CHE,Cherry Rosa,28,-0.970942,-0.239316,2024,"[[0.37274458930000004, 0.539771659, 0.0, 0.0, ...",SFB
3774,2024-07-18,V02,33,VAP,Yoyomo,0.5980,BSUF,Uva Amarilla,29,-0.935016,-0.354605,2024,"[[0.4868501577, 0.3822832708, 0.0, 0.0, 0.2966...",VAP
3784,2024-07-24,V02,8,VAP,Top 2417,0.1805,CHE,Cherry Naranja,30,-0.885456,-0.464723,2024,"[[1.10012998, 0.7647058824, 0.0, 0.0, 0.501364...",VAP


In [63]:
actuals

array([2.45300e+04, 2.11610e+04, 2.29520e+04, 1.61540e+04, 5.88800e+03,
       3.54360e+04, 1.29800e+03, 2.36000e+04, 1.62560e+04, 7.74400e+03,
       1.73920e+04, 2.33600e+03, 2.93440e+04, 5.12000e+02, 8.32000e+02,
       7.18400e+03, 6.08000e+02, 3.64800e+03, 1.53120e+04, 4.64000e+02,
       9.45000e+02, 1.12000e+03, 1.69600e+03, 1.61160e+04, 2.15560e+04,
       3.45600e+03, 1.03890e+04, 6.22900e+03, 8.16000e+02, 8.65600e+03,
       1.88800e+03, 1.46880e+04, 5.18400e+03, 2.84800e+03, 1.81120e+04,
       3.50100e+03, 1.29600e+03, 3.29600e+03, 2.51840e+04, 3.82400e+03,
       2.00000e+03, 1.60000e+01, 2.83200e+03, 1.45600e+03, 1.57120e+04,
       3.63200e+03, 2.07840e+04, 6.60800e+03, 1.40800e+03, 1.93600e+03,
       1.23200e+03, 1.42400e+03, 1.00800e+03, 7.84000e+02, 7.23200e+03,
       1.66080e+04, 8.43200e+03, 2.26880e+04, 4.64000e+03, 2.14400e+03,
       6.24000e+02, 1.09500e+04, 2.44800e+03, 8.99200e+03, 3.63200e+03,
       1.39200e+03, 8.19200e+03, 3.80800e+03, 3.04000e+02, 2.180